<a href="https://www.kaggle.com/code/giocon/gen-ai-intensive-course-capstone-2025q1?scriptVersionId=236705877" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

## Automating content creation

This project uses Gemini APIs to generate a blog post about the project itself.


The idea had its birth shortly after I had read the [competition's rules](https://www.kaggle.com/competitions/gen-ai-intensive-course-capstone-2025q1). Something about them was disturbing to me: I realized that to achieve a decent score, the blog and YouTube video multipliers were a big boost. Even though I'm obviously not aiming for the top—to me this is more a way to experiment with Gen AI (also, I could not afford to put more effort than I have already put)—I'm still kind of a competitive fellow.


The disturbing part was that I'm lazy about doing these kind of things. I mean, I love them but only if they are done by someone other than me. You get the point: let the AI do the job!

## Setup

Install the SDK and other tools for this notebook, then import the package and set up a retry policy so you don't have to manually retry when you hit a quota limit.

In [ ]:
!pip uninstall -qyy jupyterlab
!pip install -qU "google-genai==1.9.0"

In [ ]:
from google import genai
from google.genai import types

from IPython.display import Markdown, HTML

genai.__version__

### API key

To run the following cell, your API key must be stored it in a [Kaggle secret](https://www.kaggle.com/discussions/product-feedback/114053) named `GOOGLE_API_KEY`.

If you don't already have an API key, you can grab one from [AI Studio](https://aistudio.google.com/app/apikey). You can find [detailed instructions in the docs](https://ai.google.dev/gemini-api/docs/api-key).

To make the key available through Kaggle secrets, choose `Secrets` from the `Add-ons` menu and follow the instructions to add your key or enable it for this notebook.

In [ ]:
from kaggle_secrets import UserSecretsClient

GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
client = genai.Client(api_key=GOOGLE_API_KEY)

If you received an error response along the lines of `No user secrets exist for kernel id ...`, then you need to add your API key via `Add-ons`, `Secrets` **and** enable it.

![Screenshot of the checkbox to enable GOOGLE_API_KEY secret](https://storage.googleapis.com/kaggle-media/Images/5gdai_sc_3.png)

### Automated retry

Set up a retry helper. This allows you to "Run all" without worrying about per-minute quota.

In [ ]:
from google.api_core import retry

is_retriable = lambda e: (isinstance(e, genai.errors.APIError) and e.code in {429, 503})

genai.models.Models.generate_content = retry.Retry(
    predicate=is_retriable)(genai.models.Models.generate_content)

## 1. Retrieve the document

In [ ]:
import os

# Load the notebook
notebook_path = '/kaggle/input/gen-ai-intensive-course-capstone-2025q1-data/notebook.ipynb'
try:
  with open(notebook_path, 'r') as f:
      notebook = f.read()
  print("Notebook content loaded successfully from file.")

except FileNotFoundError:
    print(f"Error: File not found at {notebook_path}")
except Exception as e:
    print(f"An error occurred: {e}")

This helper function strips the notebook from the parts that are not relevant for our purpose.

In [ ]:
import json

def strip_notebook(json_string):
    """Returns the notebook's cells, with only the relevant content."""
    nb = json.loads(json_string)
    cells = [{
        "cell_type": cell["cell_type"],
        "source": cell["source"]
    } for cell in nb["cells"]]
    return json.dumps(cells)

notebook = strip_notebook(notebook)

## 2. Establish the evaluation rubric

In this part, we'll make so that our evaluations are grounded on the official source of information: the page of the competition itself. We will let the model extract the rubric for us.


Since the page loads its content dynamically, we need to simulate the access from a web browser.

In [ ]:
!pip install playwright
!playwright install chromium

In [ ]:
from playwright.async_api import async_playwright

async def get_dynamic_content(url_string, wait_for=[]):
    """
    Returns the dynamic content of a url, waiting for it to load.
    
    :param wait_for: Selectors identifying the elements required to be loaded
    """
    async with async_playwright() as p:
        browser = await p.chromium.launch()
        page = await browser.new_page()
        if wait_for:
            await page.goto(url_string, wait_until='domcontentloaded')
            for selector in wait_for:
                await page.wait_for_selector(selector)
        else:
            await page.goto(url_string, wait_until='networkidle')
        content = await page.content()
        await browser.close()
        return content

html = await get_dynamic_content(
    'https://www.kaggle.com/competitions/gen-ai-intensive-course-capstone-2025q1',
    wait_for=['#evaluation'] # We previously inspected that the rubric is mostly within the element with id="evaluation"
)

In [ ]:
prompt = ('Get the evaluation rubric from this html.\n'
          'INSTRUCTIONS\n'
          '* Be concise and schematic.\n'
          '* Add referenced information, if available and relevant.\n'
          '* Avoid introduction, comments, and final considerations.\n'
          '* Stay grounded to the source.\n')

overall_rubric = client.models.generate_content(
    model='gemini-2.0-flash',
    config=types.GenerateContentConfig(
        temperature=0
    ),
    contents=[prompt, html]).text

In [ ]:
Markdown(overall_rubric)

Now that we know the overall parameters to adhere to, let's dive into the specifics of this project: here we define a rubric for the evaluation of the final output, the generated blog.

In [ ]:
blog_rubric = """\
Poor:
    - Structure & Clarity: Lacks structure; ideas are scattered or hard to follow. Sentences may be convoluted or poorly written.
    - Technical Communication: Either extremely vague or overburdened with code-level details. Little to no explanation of core ideas.
    - Audience Awareness: Fails to consider the reader; uses unexplained jargon or alternatively avoids all technical language entirely.
    - Tone & Style: Dry or confusing. Attempts at humor (if any) feel forced or inappropriate. The text may be overly pedantic or tedious to read.
	- Added Value: Simply restates the code or project without adding perspective, insights, or broader context.
Fair:
    - Structure & Clarity: Some logical flow, but still rough or awkward. Paragraphs may lack transitions or focus.
    - Technical Communication: Describes some methods but may dwell too much on details or skip key concepts.
    - Audience Awareness: Partial awareness of audience, but either too formal or too casual. Explains some technical terms, but inconsistently.
	- Tone & Style: Attempt at readability is evident but limited. Humor is absent or not well integrated. The post may still feel dry or overly verbose.
	- Added Value: Offers a minimal layer of interpretation or commentary beyond the notebook.
Satisfactory:
    - Structure & Clarity: Clear enough to follow. Structure is recognizable (intro, body, conclusion), though it may feel mechanical.
    - Technical Communication: Balances explanation and code references reasonably well. Some depth, though not always well synthesized.
    - Audience Awareness: Moderate understanding of audience; some jargon is explained, some not.
	- Tone & Style: Professional tone with a few moments of personality. Readable, though it may still lean toward textbook-like or too cautious.
	- Added Value: Provides context, some insights, and shows effort to guide the reader through key parts of the project.
Good:
    - Structure & Clarity: Well-organized and coherent. Flows logically and transitions are smooth.
    - Technical Communication: Explains key methods and reasoning clearly without over-explaining. Good abstraction over technical details.
    - Audience Awareness: Calibrated for an informed reader. Balances clarity and depth; jargon is introduced carefully.
    - Tone & Style: Fluid and engaging. Some personality shines through. The post is easy to read and maintains reader interest.
    - Added Value: Reflects on choices, presents challenges or alternatives, and conveys the broader meaning or potential of the project.
Excellent:
    - Structure & Clarity: Exceptionally well-structured. Each section builds on the previous, guiding the reader naturally through the narrative.
    - Technical Communication: Strikes an excellent balance: technical enough to be informative, abstract enough to remain accessible.
    - Audience Awareness: Crystal-clear understanding of who the reader is. Uses analogies or narrative devices where helpful without dumbing down.
    - Tone & Style: Smooth, concise, yet expressive. Possibly includes humor or witty remarks that enhance readability and approachability.
    - Added Value: Demonstrates deep insight and reflection. Brings the project to life, goes beyond surface-level commentary, and sparks curiosity.
"""

Since we don't need the reasonig behind the assignement of the rating—the rubric is already very detailed—we will force the evaluation to the following structured output.

In [ ]:
from enum import Enum

class BlogRating(Enum):
    POOR = 'Poor'
    FAIR = 'Fair'
    SATISFACTORY = 'Satisfactory'
    GOOD = 'Good'
    EXCELLENT = 'Excellent'

## 3. Evaluate the document *(optional)*

This is not the main point of the project, but since the competition goals are pretty clear and well-structured, why not let the model help us achieving the best possible result?

In [ ]:
instructions = 'You are an expert evaluator. Your task is to rate a document, according to a provided rubric.'

prompt = """\
# INPUTS
1. The document.
2. The rubric.

# INSTRUCTIONS
1. Start with a global evaluation.
2. Highlight strengths.
3. Highlight weaknesses.
4. Briefly suggest possible improvements.
5. Give a score to the notebook only.
"""

notebook_evaluation = client.models.generate_content(
    model='gemini-2.0-flash',
    config=types.GenerateContentConfig(
        system_instruction=instructions,
        temperature=0
    ),
    contents=[prompt, notebook, overall_rubric]).text

In [ ]:
Markdown(notebook_evaluation)

## 4. Blog generation

This is the pivotal part of the project. Let's try to build the most effective prompt towards the desired result.

In [ ]:
def generate_blog():
    """Generates a blog of this notebook, based on a set of instructions and evaluation criteria."""
    instructions = 'You are a navigated blogger, passionate about coding and AI.'
    prompt ="""\
    Write me a blog post about this Kaggle notebook.
    
    INSTRUCTIONS
    - Skip the "Setup" part of the notebook.
    - Stick to the blog part of the "overall rubric".
    - Maximize the rating according to the "blog rubric".
    
    OVERALL RUBRIC
    {overall_rubric}
    
    BLOG RUBRIC
    {blog_rubric}
    """
    return client.models.generate_content(
        model='gemini-2.0-flash',
        config=types.GenerateContentConfig(
            system_instruction=instructions,
            temperature=0.2 # Raise the temperature a little bit, to enhance creativity
        ),
        contents=[prompt.format(overall_rubric=overall_rubric, blog_rubric=blog_rubric), notebook]).text

We now let the model generate the blog and then evaluate its own product. We make sure that we get only the best: if it's not up to our standards then... let's try again!


As already discussed, we're not interested in the reasoning behind the evaluation. It is just meant for the final rating, so we will enforce the structure of its output—thus also sparing precious tokens.

In [ ]:
def evaluate_blog(blog, rubric):
    """Evaluates the blog according to a rubric, and returns a summary rating."""
    prompt = """\
    You are an expert evaluator. Your task is to rate a technical blog. The result is a single rating, avaraging the rubric's criteria.
    
    # BLOG
    {blog}
    
    # RUBRIC
    {rubric}
    """
    
    return client.models.generate_content(
        model='gemini-2.0-flash',
        config=types.GenerateContentConfig(
            temperature=0,
            response_mime_type="text/x.enum",
            response_schema=BlogRating,
        ),
        contents=[prompt.format(blog=blog, rubric=rubric)]).parsed

max_tries = 10
tryn = 0
blog = generate_blog()
blog_evaluation = evaluate_blog(blog, blog_rubric)
blog_backup = None
while blog_evaluation is not BlogRating.EXCELLENT:
    print("Discarded blog rated: {}.".format(blog_evaluation))
    if blog_evaluation is BlogRating.GOOD:
        blog_backup = blog
    tryn += 1
    if tryn == max_tries:
        break
    blog = generate_blog()
    blog_evaluation = evaluate_blog(blog, blog_rubric)

# If no 'Excellent' blog was generated, backup to a 'Good' blog if possible
if tryn == max_tries:
    if blog_backup is None:
        raise Exception('Gave up generating a blog: quality is too poor...')
    else:
        print("Recovered last 'Good' blog.".format(blog_evaluation))
        blog = blog_backup

In [ ]:
Markdown(blog)

Save the blog to the output folder.

In [ ]:
with open("blog.md", "w", encoding="utf-8") as f:
    f.write(blog)

## Final thoughts

And this is the end of the project. To me, it has been a very enjoyable journey. But, before ending, I want to briefly discuss about the following.

### Things that didn't work too well

Initially I tried to use the Google search grounding feature to retrieve some required web content (the information in the competion page). But, honestly, the results that I got were not satisfactory. I had to opt for using the playwright library instead.

### Other applications

The fun part of the project is this form of self-reflection, where the code that conjures the AI is then used and evaluated by the conjured AI itself. But of course, the fundamental steps (1. Retrieve the document, 2. Establish the evaluation rubric, 3. Blog generation) are easily abstracted and generalized to other applications. For instance, to write a blog/documentation of another given document.

### Further directions

With the same logic, I would have loved to use Gen AI to produce also the YouTube video about the project. But, other than I could not afford it in terms of time, the task is evidently more complex than the blog creation. At the time of writing, as far as I know there is not a Gemini model well suited to video generation (at least a free of charge one). I was thinking to overcome this difficulty by doing something like a slide show: let the AI produce an audio file and a set of images, which are then to be assembled in a second moment. The not easy part, in my mind, is to make the correct prompts so that the generated files are coherent with each other. But I suppose that I will reserve this idea for the next time.

Thank you so much for your attention. Bye!